In [18]:
library(glmnet)
nrows <- 8
ncols <- 5
set.seed(1234)
X0 <- matrix(rnorm(nrows*ncols), nrows, ncols)
Sigmainv <- .25^abs(outer(1:ncols,1:ncols,"-"))
X <- backsolve( chol(Sigmainv), X0)
X0

-1.2070657,-0.56445200,-0.5110095,-0.6937202,-0.7094400
0.2774292,-0.89003783,-0.9111954,-1.4482049,-0.5012581
1.0844412,-0.47719270,-0.8371717,0.5747557,-1.6290935
-2.3456977,-0.99838644,2.4158352,-1.0236557,-1.1676193
0.4291247,-0.77625389,0.1340882,-0.0151383,-2.1800396
0.5060559,0.06445882,-0.4906859,-0.9359486,-1.3409932
-0.5747400,0.95949406,-0.4405479,1.1022975,-0.2942939
-0.5466319,-0.11028549,0.4595894,-0.4755931,-0.4658975


In [19]:
X

-1.278697672,-0.3346452,-0.2757399,-0.31979535,-0.58001576
0.006526181,-0.7960165,-0.7249218,-1.64410089,-0.09706697
1.725662574,-0.2350602,-1.4883932,0.85791193,-1.38104250
-2.533425688,-0.8307012,2.4604424,-1.05331839,-0.64302817
0.443198073,-0.8017116,0.1384857,-0.01563477,-2.25153527


In [24]:
lambda = rep(0,ncol(X))
for (k in 1:ncol(X)) {
    a = 2 * (ncol(X)-k+1) / ncol(X)
    lambda[k] = a*sqrt(var(X[,k])*log(ncol(X))/nrow(X))
    }

lambda

[1] 1.8551532 0.2630657 1.0148239 0.4368239 0.1908930

In [25]:
for (k in 1:ncol(X)) {
fitreg  = glmnet::glmnet(X[,-k],X[,k], family="gaussian",
                         lambda = lambda,
                             standardize = FALSE)
print(fitreg$beta[1])
}

[1] 0
[1] 0
[1] 0
[1] 0
[1] 0


In [46]:
PrecXia = function(X){
  n = nrow(X)
  betahat = matrix(0,ncol(X)-1,ncol(X))
  reshat  = X
  for (k in 1:ncol(X)){
    fitreg  = glmnet::glmnet(X[,-k],X[,k],family="gaussian",
        lambda = 2*sqrt(var(X[,k])*log(ncol(X))/nrow(X)),
                             standardize = FALSE) 
    betahat[,k] = as.vector(fitreg$beta)
    reshat[,k]  = X[,k]-as.vector(predict(fitreg,X[,-k]))
  }
  rtilde = cov(reshat)*(n-1)/n
  rhat   = rtilde
  for (i in 1:(ncol(X)-1)){
    for (j in (i+1):ncol(X)){
      rhat[i,j] = -(rtilde[i,j]+rtilde[i,i]*betahat[i,j]+rtilde[j,j]*betahat[j-1,i])
      rhat[j,i] = rhat[i,j]
    }
  }
  Tprec  = 1/rhat
  TprecStd = Tprec
  for (i in 1:(ncol(X)-1)){
    for (j in (i+1):ncol(X)){
      Tprec[i,j] = Tprec[j,i] = rhat[i,j]/(rhat[i,i]*rhat[j,j])
      thetahatij = (1+(betahat[i,j]^2*rhat[i,i]/rhat[j,j]))/(n*rhat[i,i]*rhat[j,j])   
      TprecStd[i,j] = TprecStd[j,i] = Tprec[i,j]/sqrt(thetahatij)
    }
  }
  
  MatPrecXia = list(Tprec=Tprec,TprecStd=TprecStd)
}

In [23]:
result <- PrecXia(X)

In [24]:
result

0.9329074,-0.2050439,0.7263450,-0.3114785,0.6028658
-0.2050439,2.7587035,0.5276542,-1.3085151,-1.1540177
0.7263450,0.5276542,0.9651949,0.2950095,0.1728215
-0.3114785,-1.3085151,0.2950095,1.5349390,0.1808986
0.6028658,-1.1540177,0.1728215,0.1808986,2.6327341
0.9329074,-0.3615098,2.1650162,-0.7362210,1.0880350
-0.3615098,2.7587035,0.9146066,-1.7985631,-1.2111599
2.1650162,0.9146066,0.9651949,0.6855323,0.3066421
-0.7362210,-1.7985631,0.6855323,1.5349390,0.2545254
1.0880350,-1.2111599,0.3066421,0.2545254,2.6327341


In [25]:
var(X[,1])

[1] 1.225049